In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bitsandbytes # 모델을 경량화해서 로드
!pip install transformers
!pip install peft # LoRA 튜닝을 하게 해주는 라이브러리
!pip install accelerate
!pip install trl
!pip install datasets

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Gemma2ForCausalLM
from datasets import Dataset
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from trl import SFTTrainer
import os
import re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/train.csv')
train

,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."
...,...,...,...,...,...
491,TRAIN_491,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,자치단체 보조금과 민간보조금은 각각 어떤 비율로 증가했는가?,"자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10..."
492,TRAIN_492,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,한국의 재정금융안정계획이 주로 어떤 목표에 초점을 맞추어 있었는가?,한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
493,TRAIN_493,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,1952년에 체결된 '한미경제조정협정'은 어떤 문제를 해결하기 위해 체결되었는가?,"원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ..."
494,TRAIN_494,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,"2023년 12월 IMF는 성장 전망을 어떻게 제시하고 있으며, 성장세의 둔화가 어...","IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ..."


In [ ]:
def clean_text(text):
    cleaned_text = re.sub(r'[①②③④⑤⑥⑦⑧⑨⑩=\-\n]', '', text)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

In [ ]:
train['Answer'] = train['Answer'].apply(clean_text)

In [ ]:
model_id = "rtzr/ko-gemma-2-9b-it"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False
model = Gemma2ForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        attn_implementation='eager',
        trust_remote_code=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm()
        (post_attention_layernorm): Gemma2RMSNorm(

In [ ]:
dataset = []
for question, answer in tqdm(zip(train['Question'], train['Answer']), total = len(train)):
    dataset.append(f"### 질문 : {question}\n\n질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.\n\n### 답변: {answer}\n"+tokenizer.eos_token)
dataset = Dataset.from_dict({"text": dataset})
dataset = dataset.shuffle()
dataset

  0%|          | 0/496 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 496
})

In [ ]:
print(dataset['text'][0])

### 질문 : 성과계획서는 어떤 목적을 위해 작성되는가?

질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.

### 답변: 성과계획서는 중앙관서 및 기금관리주체의 전략목표 및 성과목표를 달성하기 위한 연도별 시행계획을 작성하기 위해 사용된다.
<eos>


In [ ]:
token_length = []
for i in range(len(dataset)):
  token_length.append(len(tokenizer.encode(dataset['text'][i])))
pd.Series(token_length, name = 'token_length').describe()

,token_length
count,496.000000
mean,138.558468
std,42.233366
min,78.000000
25%,113.000000
50%,129.000000
75%,151.250000
max,523.000000


In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config_dict = {
    'lora_r' : 8,
    'lora_alpha' : 32,
    'lora_dropout' : 0.05,
    'lora_bias' : "none",
    'lora_task_type' : "CAUSAL_LM",
    'target_modules': ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"]
}

peft_config = LoraConfig(
            lora_alpha=lora_config_dict['lora_alpha'],
            lora_dropout=lora_config_dict['lora_dropout'],
            r=lora_config_dict['lora_r'],
            bias=lora_config_dict['lora_bias'],
            task_type=lora_config_dict['lora_task_type'],
            target_modules = lora_config_dict['target_modules']
        )

In [ ]:
output_dir = "/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/pymupdf4llm/q&a_finetune_cleaned/"

In [ ]:
train_param = {
    'num_train_epochs': 5,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "adamw_torch",
    'save_steps': 300,
    'logging_steps': 10,
    'learning_rate': 0.0002,
    'weight_decay':  0.01,
    'max_grad_norm': 1,
    'warmup_ratio': 0.06,
    'group_by_length' : False,
    'lr_scheduler_type': 'cosine'
}

training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=train_param['num_train_epochs'],
            per_device_train_batch_size=train_param['per_device_train_batch_size'],
            per_device_eval_batch_size=train_param['per_device_eval_batch_size'],
            gradient_accumulation_steps=train_param['gradient_accumulation_steps'],
            optim=train_param['optim'],
            save_steps=train_param['save_steps'],
            logging_steps=train_param['logging_steps'],
            learning_rate=train_param['learning_rate'],
            weight_decay=train_param['weight_decay'],
            # fp16=kwargs['fp16'],
            # bf16=kwargs['bf16'],
            max_grad_norm=train_param['max_grad_norm'],
            # max_steps=kwargs['max_steps'],
            warmup_ratio=train_param['warmup_ratio'],
            group_by_length=train_param['group_by_length'],
            lr_scheduler_type=train_param['lr_scheduler_type'],
#             report_to="tensorboard"
        )

In [ ]:
trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,
            peft_config=peft_config,
            dataset_text_field="text",
            max_seq_length=530,
            tokenizer=tokenizer,
            args=training_args,
            packing=False,
            # neftune_noise_alpha=5,
            # callbacks=[AbortCallback()]
            )

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [ ]:
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)
# trainer.train(resume_from_checkpoint=True)

Step,Training Loss
10,2.332000
20,1.394400
30,1.193900
40,0.945200
50,1.077300
60,1.012800
70,0.930800
80,1.078300
90,1.115600
100,1.044200


TrainOutput(global_step=992, training_loss=0.7207032313510295, metrics={'train_runtime': 953.9108, 'train_samples_per_second': 1.04, 'train_steps_per_second': 1.04, 'total_flos': 6887243718297600.0, 'train_loss': 0.7207032313510295, 'epoch': 2.0})

In [ ]:
trainer.model.save_pretrained(output_dir)